### Connect:

In [ ]:
import phywhisperer.usb as pw
phy = pw.Usb()
phy.con()
phy.set_power_source("host")

### Test register R/W:

In [ ]:
phy.usb.cmdWriteMem(0, [0xa2])
assert (phy.usb.cmdReadMem(0, 1)[0] == 0xa2), "readback error!"

### Reset FPGA
Before running the cells that follow, unplug target, do the reset_fpga() below, then reconnect target.

In [ ]:
phy.reset_fpga()

### Enable capture

In [ ]:
phy.usb.cmdWriteMem(3, [1])
# uncomment to disable timestamp collection:
#phy.usb.cmdWriteMem(2, [1])

### Read from FIFO:

In [ ]:
bytes_to_read = 20
fifo_bytes = []
raws = []
data_commands = 0
stat_commands = 0
total_time = 0
while (data_commands < bytes_to_read):
    raw = phy.usb.cmdReadMem(1,3)
    raws.append(raw)
    command = raw[2] & 0x3
    if (command == 0):
        data = raw[1]
        ts = raw[0] & 0x7
        flags = (raw[0] & 0xf8) >> 3
        total_time += ts
        print ("DATA: data=%s, flags=%s, time=%d" % (hex(data), hex(flags), total_time))
        data_commands += 1
        total_time = 0
        fifo_bytes.append(data)
    elif (command == 1):
        ts = raw[0] & 0x7
        flags = (raw[0] & 0xf8) >> 3
        total_time += ts
        print ("STAT: flags=%s, time=%d" % (hex(flags), total_time))
        stat_commands += 1
        total_time = 0
    elif (command == 2):
        ts = raw[0] + (raw[1] << 8)
        total_time += ts
        #print ("TIME: time=%d" % ts)
    else:
        print ("ERROR: unknown command (%d)" % command)        